In [1]:
import warnings
warnings.filterwarnings("ignore")
from evaluation.mol_structure import list_of_smiles_to_nx_graphs,list_of_nx_graphs_to_smiles
from evaluation.mol_structure import draw_one_mol, draw_graphs
from evaluation.new_structural_metric import symmetric_graph_set_distance,atom,cycle,neighborhood
from evaluate import evaluate
from evaluation.moses.metrics.metrics import get_all_metrics
#import utils.graph_generators as gen
import torch
import networkx as nx
import os 
import sys
import json
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)
import rdkit
import numpy as np
from evaluation.utils import get_data,  get_mock_data,remove_empty_graphs_and_targets,\
    get_generated_data,preprocess,get_graph_data, get_generated_graph_data


In [17]:
generated_graphs=pd.read_pickle('/Users/cosmina/Documents/GGM-metrics/data/fb/corrupted/train1/ames_50/ames_50_corrupted_with_0.pkl')

# Perturbed

In [39]:
import pandas as pd
def get_corrupted_graph_data(name,percent ,return_smiles=False):
   try:
     generated_graphs=pd.read_pickle(f'data/fb/corrupted/train1/{name}/{name}_corrupted_with_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/fb/corrupted/train1/{name}/{name}_perturbed_with_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
        for n,i in enumerate(g.nodes(data=True)):
          try:
            g.nodes[n]['label']=int(g.nodes[n]['label'])
          except: None
          try:
           g.nodes[n]['true_label']=int(g.nodes[n]['true_label'])
          except: None
          try:
             g.nodes[n]['attr']=[int(l) for l in list(g.nodes[n]['attr'])]
          except: None
          try:
           del g.nodes[n]['target']
          except:None
   if return_smiles:
      return(list_of_nx_graphs_to_smiles(generated_graphs))
   return  generated_graphs, np.array(targets)


def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
      try:
        mean_dict[key] = sum(float(d[key]) for d in dict_list) / len(dict_list)
      except: mean_dict[key]=dict_list[0][key]
    return mean_dict

## Molecular graphs 

### nn and structural

In [7]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_corrupted_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)


ames_50 corrupted
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
{'activations_time': 0.622242800394694, 'fid': 102.26703535873706, 'fid_time': 0.8317867120107015, 'kid': 20234.503577471896, 'kid_time': 0.7596324443817138, 'precision': 0.973354273000916, 'recall': 0.9738254155215286, 'f1_pr': 0.9735958836801167, 'precision_time': 5.8928834915161135, 'recall_time': 5.8928834915161135, 'f1_pr_time': 5.8928834915161135, 'density': 0.9970946211228896, 'coverage': 0.9685381494568774, 'f1_dc': 0.9826010804643743, 'density_time': 4.1197923501332605, 'coverage_time': 4.1197923501332605, 'f1_dc_time': 4.1197923501332605, 'mmd_rbf': 0.000835327006643638, 'mmd_rbf_time': 6.7939804236094155, 'mmd_linear': 0.11458013877272606, 'mmd_line

### molecular metrics

In [4]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    print(dataset_name+split, generator)
                    graphs, splits  =get_data(dataset_name+split, return_smiles=True)
                    generated_smiles=get_corrupted_graph_data(dataset_name+split,int(percent*100),return_smiles=True)
                    train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                    metrics=get_all_metrics(gen=generated_smiles,train=train2_smiles,test=train2_smiles)
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                    break
            
            break
        break

ames_50 corrupted
{'valid': 0.8044758539458186, 'unique@1000': 0.989, 'unique@10000': 0.9760858955588092, 'FCD/Test': 3.0052759420133413, 'SNN/Test': 0.5673616151218591, 'Frag/Test': 0.982201276316404, 'Scaf/Test': 0.8070805773197628, 'FCD/TestSF': 22.345977974563844, 'SNN/TestSF': 0.34160188013908627, 'Frag/TestSF': 0.7335194815083179, 'Scaf/TestSF': 0.16624877929197746, 'IntDiv': 0.9126627980092435, 'IntDiv2': 0.8931481730218955, 'Filters': 0.6676427525622255, 'logP': 0.0905483273405797, 'SA': 0.031235811756828366, 'QED': 0.01630887374196703, 'weight': 7.451398099334284, 'Novelty': 0.9695, 'dataset': 'ames_50', 'generator_name': 'corrupted', 'degree_of_perturbation': 0}


## Synthetic 

In [4]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[0]#,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in ['synthetic_2','synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, percent)
                    train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                    generated_graphs, generated_targets= get_corrupted_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)

synthetic_2_50 0
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.82495 0.82805 0.8065000000000001 0.82805
{'activations_time': 0.16185917854309081, 'fid': 0.41960447059632244, 'fid_time': 0.44933629035949707, 'kid': 0.006225705146789551, 'kid_time': 0.47835981051127113, 'precision': 0.9764999999999999, 'recall': 0.9818333333333332, 'f1_pr': 0.9791318571525486, 'precision_time': 2.32598082224528, 'recall_time': 2.32598082224528, 'f1_pr_time': 2.32598082224528, 'density': 0.9739666666666663, 'coverage': 0.9456666666666664, 'f1_dc': 0.9584439641491177, 'density_time': 1.550954802831014, 'coverage_time': 1.550954802831014, 'f1_dc_time': 1.550954802831014, 'mmd_rbf': 0.005624789247910182, 'mmd_rbf_time': 2.303980525334676, 'mm

In [11]:
train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)


In [8]:
g[0].nodes(data=True)

NodeDataView({0: {'label': 6, 'attr': [5, 0, 3, 5, 1, 0, 1, 1, 1], 'label_name': 'C'}, 1: {'label': 6, 'attr': [5, 0, 3, 5, 1, 0, 1, 1, 1], 'label_name': 'C'}, 2: {'label': 6, 'attr': [5, 0, 3, 5, 0, 0, 1, 1, 1], 'label_name': 'C'}, 3: {'label': 6, 'attr': [5, 0, 3, 5, 1, 0, 1, 1, 1], 'label_name': 'C'}, 4: {'label': 6, 'attr': [5, 0, 3, 5, 1, 0, 1, 1, 1], 'label_name': 'C'}, 5: {'label': 6, 'attr': [5, 0, 3, 5, 0, 0, 1, 1, 1], 'label_name': 'C'}, 6: {'label': 6, 'attr': [5, 0, 3, 5, 1, 0, 1, 1, 1], 'label_name': 'C'}, 7: {'label': 6, 'attr': [5, 0, 3, 5, 1, 0, 1, 1, 1], 'label_name': 'C'}, 8: {'label': 6, 'attr': [5, 0, 3, 5, 1, 0, 1, 1, 1], 'label_name': 'C'}, 9: {'label': 6, 'attr': [5, 0, 3, 5, 0, 0, 1, 1, 1], 'label_name': 'C'}, 10: {'label': 7, 'attr': [6, 0, 2, 5, 0, 0, 1, 1, 1], 'label_name': 'N'}, 11: {'label': 6, 'attr': [5, 0, 3, 5, 1, 0, 1, 1, 1], 'label_name': 'C'}, 12: {'label': 6, 'attr': [5, 0, 3, 5, 0, 0, 1, 1, 1], 'label_name': 'C'}, 13: {'label': 6, 'attr': [5, 0, 3,

In [12]:
len(g),len(train2_graphs)

(2547, 2547)

# Mode Collapse

In [40]:

import pandas as pd
def get_mode_collapse_graph_data(name,percent,return_smiles=False):
   try:
     generated_graphs=pd.read_pickle(f'data/fb/mode_collapse/train1/{name}/{name}_no_of_clusters_collapsed_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/fb/mode_collapse/train1/{name}/{name}_fraction_of_clusters_collapsed_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
      try:
        target= int(g.nodes[0]['target'])
        targets.append(target)
      except:None
      
      for n,i in enumerate(g.nodes(data=True)):
        try:
          g.nodes[n]['label']=int(g.nodes[n]['label'])
        except: None
        try:
          g.nodes[n]['true_label']=int(g.nodes[n]['true_label'])
        except: None
        try:
            g.nodes[n]['attr']=[int(l) for l in list(g.nodes[n]['attr'])]
        except: None
        try:  del g.nodes[n]['target']
        except:None
   if return_smiles:
      return(list_of_nx_graphs_to_smiles(generated_graphs))
   return  generated_graphs, np.array(targets)

## Molecular Graphs

### NN and structural

In [12]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['mode_collapse_v_2']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)


ames_50 mode_collapse_v_2
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.38976160685221356, 'fid': 137.17303041018678, 'fid_time': 0.5103121439615885, 'kid': 127.25725270907084, 'kid_time': 0.5007567246754964, 'precision': 0.9712603062426383, 'recall': 0.9732757492474807, 'f1_pr': 0.9722656536693334, 'precision_time': 5.4426788806915285, 'recall_time': 5.4426788806915285, 'f1_pr_time': 5.4426788806915285, 'density': 0.9972254940452819, 'coverage': 0.9688784190550975, 'f1_dc': 0.982838398799357, 'density_time': 3.675448497136434, 'coverage_time': 3.675448497136434, 'f1_dc_time': 3.675448497136434, 'mmd_rbf': 0.0008066629952130218, 'mmd_rbf_time': 5.920201524098714, 'mmd_linear': 0.17394636174043018, 'mmd_linear_time': 0.39075209299723307, 'wl_mmd': 0.0004067609503843883, 'wl_mmd_time': 3.319516897201538, 'nspdk_mmd': 0.0006410119553881266, 'nspdk_mmd_time': 17.76198983192444, 'degree_mmd': 3.204833337711044e-05, 'degree_mmd_time': 76

IndexError: list index out of range

In [41]:
current_smiles_datasets=['ames']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[1]
metrics={}
for generator in ['mode_collapse_v_2']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)

ames_50 mode_collapse_v_2
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.21271756490071614, 'fid': 7.7531114249818875, 'fid_time': 0.2818642775217692, 'kid': 0.04845168087631464, 'kid_time': 0.2981719652811686, 'precision': 0.9785714285714284, 'recall': 0.9128386336866903, 'f1_pr': 0.9444724799476224, 'precision_time': 0.9929906050364177, 'recall_time': 0.9929906050364177, 'f1_pr_time': 0.9929906050364177, 'density': 0.9511904761904763, 'coverage': 0.08590498625834316, 'f1_dc': 0.15758467253201086, 'density_time': 0.8748451709747315, 'coverage_time': 0.8748451709747315, 'f1_dc_time': 0.8748451709747315, 'mmd_rbf': 0.04612797796726227, 'mmd_rbf_time': 1.0034213542938233, 'mmd_linear': 2.5470896005630492, 'mmd_linear_time': 0.21296885808308919, 'wl_mmd': 0.04153927686671577, 'wl_mmd_time': 0.8699710369110107, 'nspdk_mmd': 0.060521248785171344, 'nspdk_mmd_time': 8.008897066116333, 'degree_mmd': 0.007575709313989876, 'degree_mmd_time': 

In [45]:
current_smiles_datasets=['bbb_martins']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['mode_collapse_v_2']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)

bbb_martins_50 mode_collapse_v_2
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.15779336293538412, 'fid': 13.308638771829553, 'fid_time': 0.18706541061401366, 'kid': 7.469306929906209, 'kid_time': 0.18839608828226725, 'precision': 0.9740845070422536, 'recall': 0.9499530516431924, 'f1_pr': 0.9618560932784562, 'precision_time': 0.6253319263458252, 'recall_time': 0.6253319263458252, 'f1_pr_time': 0.6253319263458252, 'density': 1.0410516431924883, 'coverage': 0.9628169014084507, 'f1_dc': 1.0003322187482326, 'density_time': 0.8977382977803549, 'coverage_time': 0.8977382977803549, 'f1_dc_time': 0.8977382977803549, 'mmd_rbf': 0.0032195281858245534, 'mmd_rbf_time': 1.306471840540568, 'mmd_linear': 0.5394686222076416, 'mmd_linear_time': 0.1578603744506836, 'wl_mmd': 0.0008432410168706106, 'wl_mmd_time': 0.8034179210662842, 'nspdk_mmd': 0.002491165211787272, 'nspdk_mmd_time': 5.154574155807495, 'degree_mmd': 3.895604203107972e-05, 'degree_mmd

In [46]:
current_smiles_datasets=['cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['mode_collapse_v_2']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)

cyp1a2_veith_50 mode_collapse_v_2
Now computing classifier based metrics
fid calculation produces singular product; adding 1e-06 to diagonal of cov estimates
Now computing structural based metrics
{'activations_time': 0.3300670305887858, 'fid': 1.739139078834586, 'fid_time': 0.33481953938802084, 'kid': 2.599631245682637, 'kid_time': 0.4628208955128988, 'precision': 0.9635537377868666, 'recall': 0.9642051049004013, 'f1_pr': 0.9638870687672259, 'precision_time': 1.3703811645507813, 'recall_time': 1.3703811645507813, 'f1_pr_time': 1.3703811645507813, 'density': 1.0073437855032947, 'coverage': 0.9682041960160569, 'f1_dc': 0.9873719108685669, 'density_time': 0.9402279059092203, 'coverage_time': 0.9402279059092203, 'f1_dc_time': 0.9402279059092203, 'mmd_rbf': 0.000455135878701185, 'mmd_rbf_time': 2.736153825124105, 'mmd_linear': 0.021244437557955582, 'mmd_linear_time': 0.3302517255147298, 'wl_mmd': 0.00010821777191694082, 'wl_mmd_time': 5.786354064941406, 'nspdk_mmd': 0.00033186027571008303,

### Molecular metrics

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['mode_collapse_v_2']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    print(dataset_name+split, generator)
                    graphs, splits  =get_data(dataset_name+split, return_smiles=True)
                    generated_smiles=get_mode_collapse_graph_data(dataset_name+split,int(percent*100),return_smiles=True)
                    train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                    metrics=get_all_metrics(gen=generated_smiles,train=train2_smiles,test=train2_smiles)
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)

## Synthetic graphs

In [42]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['mode_collapse_v_2']:
    for dataset_name in ['synthetic_2','synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, percent)
                    train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                    generated_graphs, generated_targets= get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)

synthetic_2_50 0
Now computing structural based metrics
Now computing the auc_roc based  metric
0.82495 0.82805 0.8065000000000001 0.82805
{'activations_time': 0.0390959898630778, 'fid': 0.31363409003723886, 'fid_time': 0.041717116038004556, 'kid': 0.005695184071858724, 'kid_time': 0.06332470575968424, 'precision': 0.9775000000000003, 'recall': 0.9826666666666667, 'f1_pr': 0.9800583581202882, 'precision_time': 0.10608439445495606, 'recall_time': 0.10608439445495606, 'f1_pr_time': 0.10608439445495606, 'density': 0.9577000000000003, 'coverage': 0.9468333333333331, 'f1_dc': 0.9516960031034024, 'density_time': 0.0782965342203776, 'coverage_time': 0.0782965342203776, 'f1_dc_time': 0.0782965342203776, 'mmd_rbf': 0.005608976415048043, 'mmd_rbf_time': 0.1237455685933431, 'mmd_linear': 0.13467245598634084, 'mmd_linear_time': 0.03914831479390462, 'wl_mmd': 0.0016914465204018736, 'wl_mmd_time': 0.4098658561706543, 'nspdk_mmd': 0.003675417479218901, 'nspdk_mmd_time': 6.2905333042144775, 'degree_mm

# Overfitting 

In [43]:
import pandas as pd
def get_mode_dropping_graph_data(name,percent,return_smiles=False):
   try:
     generated_graphs=pd.read_pickle(f'data/fb/mode_dropping/train1/{name}/{name}_no_of_clusters_dropped_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/fb/mode_dropping/train1/{name}/{name}_fraction_of_clusters_dropped_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
      try:
        target= int(g.nodes[0]['target'])
        targets.append(target)
      except:None
      
      for n,i in enumerate(g.nodes(data=True)):
        try:
          g.nodes[n]['label']=int(g.nodes[n]['label'])
        except: None
        try:
          g.nodes[n]['true_label']=int(g.nodes[n]['true_label'])
        except: None
        try:
            g.nodes[n]['attr']=[int(l) for l in list(g.nodes[n]['attr'])]
        except: None
        try:  del g.nodes[n]['target']
        except:None
   if return_smiles:
      return(list_of_nx_graphs_to_smiles(generated_graphs))
   return  generated_graphs, np.array(targets)

def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
      try:
        mean_dict[key] = sum(float(d[key]) for d in dict_list) / len(dict_list)
      except: mean_dict[key]=dict_list[0][key]
    return mean_dict

## Molecular graphs

### NN and structural

In [7]:
g,t=get_mode_dropping_graph_data(dataset_name+split,int(percent*100))

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['overfitting']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_dropping_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)


### Molecular metrics

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['overfitting']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    print(dataset_name+split, generator)
                    graphs, splits  =get_data(dataset_name+split, return_smiles=True)
                    generated_smiles=get_mode_dropping_graph_data(dataset_name+split,int(percent*100),return_smiles=True)
                    train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                    metrics=get_all_metrics(gen=generated_smiles,train=train2_smiles,test=train2_smiles)
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)


ames_50 corrupted
{'valid': 0.8044758539458186, 'unique@1000': 0.989, 'unique@10000': 0.9760858955588092, 'FCD/Test': 3.0052759420133413, 'SNN/Test': 0.5673616151218591, 'Frag/Test': 0.982201276316404, 'Scaf/Test': 0.8070805773197628, 'FCD/TestSF': 22.345977974563844, 'SNN/TestSF': 0.34160188013908627, 'Frag/TestSF': 0.7335194815083179, 'Scaf/TestSF': 0.16624877929197746, 'IntDiv': 0.9126627980092435, 'IntDiv2': 0.8931481730218955, 'Filters': 0.6676427525622255, 'logP': 0.0905483273405797, 'SA': 0.031235811756828366, 'QED': 0.01630887374196703, 'weight': 7.451398099334284, 'Novelty': 0.9695, 'dataset': 'ames_50', 'generator_name': 'corrupted', 'degree_of_perturbation': 0}
ames_50 corrupted
{'valid': 0.804552590266876, 'unique@1000': 0.976, 'unique@10000': 0.9751219512195122, 'FCD/Test': nan, 'SNN/Test': 0.567236070095039, 'Frag/Test': 0.9821772820574496, 'Scaf/Test': 0.8065613886070349, 'FCD/TestSF': 22.328786817127863, 'SNN/TestSF': 0.34175933800456, 'Frag/TestSF': 0.7331510753864865,

: 

## Synthetic graphs

In [44]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
splits=['_50']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['overfitting']:
    for dataset_name in ['synthetic_2','synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, percent)
                    train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                    generated_graphs, generated_targets= get_mode_dropping_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)

synthetic_2_50 0
Now computing structural based metrics
Now computing the auc_roc based  metric
0.82495 0.82805 0.8065000000000001 0.82805
{'activations_time': 0.034383201599121095, 'fid': 0.35544029911533054, 'fid_time': 0.057544978459676106, 'kid': 0.005713351567586263, 'kid_time': 0.07916358311971029, 'precision': 0.9728333333333334, 'recall': 0.9836666666666666, 'f1_pr': 0.9782045397464041, 'precision_time': 0.22582693099975587, 'recall_time': 0.22582693099975587, 'f1_pr_time': 0.22582693099975587, 'density': 0.9642666666666667, 'coverage': 0.9471666666666666, 'f1_dc': 0.9543393668294072, 'density_time': 0.14864053726196289, 'coverage_time': 0.14864053726196289, 'f1_dc_time': 0.14864053726196289, 'mmd_rbf': 0.005732094659470022, 'mmd_rbf_time': 0.20934710502624512, 'mmd_linear': 0.13525876104831697, 'mmd_linear_time': 0.0344235897064209, 'wl_mmd': 0.0016914465204012075, 'wl_mmd_time': 0.31865596771240234, 'nspdk_mmd': 0.003675417479218901, 'nspdk_mmd_time': 6.157788038253784, 'degr